In [1]:
import torch
import pandas as pd
import numpy as np
import selfies as sf
import wandb
import os

# GPU 확인
print(f"PyTorch Version: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"Device Name: {torch.cuda.get_device_name(0)}")
else:
    print("⚠️ GPU가 감지되지 않았습니다. PyTorch 설치를 확인해주세요.")

# 경로 설정 (Clone 받은 폴더 내부의 데이터 경로)
# 논문에 따르면 subLINCS.csv를 사용합니다[cite: 27].
# 경로가 다를 경우 실제 clone된 폴더 구조를 보고 수정이 필요할 수 있습니다.
DATA_PATH = './TransGEM/data/subLINCS.csv'

PyTorch Version: 2.7.1+cu118
CUDA Available: True
Device Name: NVIDIA RTX A5000


In [2]:
def tenfold_binary_embedding(value: float):
    """
    논문의 Eq. 12, 13, 14를 구현한 Tenfold Binary Embedding 함수입니다.
    실수형 유전자 발현값을 10비트 바이너리 벡터로 변환합니다 [cite: 91-105].
    """
    # 1. Sign bit (부호 비트): 양수면 1, 아니면 0 [cite: 94]
    sign_bit = 1 if value > 0 else 0
    
    # 2. 절댓값에 10을 곱하고 정수화 [cite: 96]
    # 예: 7.3 -> 73, -14.4 -> 144
    tenfold_value = int(abs(value) * 10)
    
    # 3. 이진수 변환 및 패딩 (9 bits) [cite: 98-100]
    # 1024(2^10)를 넘지 않는다고 가정하고 9비트로 맞춤
    binary_str = bin(tenfold_value)[2:] # '0b' 제거
    padded_binary_str = binary_str.zfill(9)
    
    # 4. 벡터화 [cite: 102]
    binary_vector = [int(bit) for bit in padded_binary_str]
    
    # 5. 부호 비트와 결합 (최종 10차원 벡터) [cite: 104]
    final_embedding = [sign_bit] + binary_vector
    
    return torch.tensor(final_embedding, dtype=torch.float32)

# --- 테스트 (발표 자료의 예시 7.3과 -14.4로 검증) [cite: 107] ---
test_val1 = 7.3
test_val2 = -14.4

emb1 = tenfold_binary_embedding(test_val1)
emb2 = tenfold_binary_embedding(test_val2)

print(f"Value: {test_val1} -> Embedding: {emb1.tolist()}")
print(f"Value: {test_val2} -> Embedding: {emb2.tolist()}")
# 논문 예시와 일치하는지 확인 (7.3 -> [1, 0, 0, 1, 0, 0, 1, 0, 0, 1]) [cite: 118]

Value: 7.3 -> Embedding: [1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0]
Value: -14.4 -> Embedding: [0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]


In [3]:
if os.path.exists(DATA_PATH):
    df = pd.read_csv(DATA_PATH)
    print(f"데이터 로드 성공! Shape: {df.shape}")
    display(df.head())
    
    # 데이터 컬럼 확인 (Cell Line, SMILES, Gene Expressions 등)
    print("\nColumns:", df.columns.tolist())
else:
    print(f"❌ 파일을 찾을 수 없습니다: {DATA_PATH}")
    print("현재 경로의 파일 목록:")
    print(os.listdir('.'))
    # TransGEM 폴더 내부도 확인
    if os.path.exists('./TransGEM'):
        print("TransGEM 폴더 내부:", os.listdir('./TransGEM'))

데이터 로드 성공! Shape: (27378, 6)


,cell_line,dose,drug_id,smiles,selfies,gene_e
0,MCF7,10 uM,BRD-A60070924,CC12CCC3C(CCc4cc(O)ccc34)C1CCC2O,"['[C]', '[C]', '[C]', '[C]', '[C]', '[C]', '[B...",-0.1//-0.3//-0.2//0.1//0.1//-0.1//-0.2//0.0//0...
1,MCF7,10 uM,BRD-K71879491,CC(/C=C/C1=C(C)CCCC1(C)C)=CC=CC(C)=CC(O)=O,"['[C]', '[C]', '[Branch2_1]', '[Ring1]', '[C]'...",0.3//0.1//0.1//0.0//-0.3//-0.4//0.2//-0.3//0.2...
2,MCF7,10 uM,BRD-K43797669,Oc1ccc(cc1)-c1coc2cc(O)cc(O)c2c1=O,"['[O]', '[C]', '[=C]', '[C]', '[=C]', '[Branch...",0.0//0.1//-0.4//0.0//0.2//-0.1//-0.1//-0.1//-0...
3,MCF7,10 uM,BRD-A13084692,Cc1c(C)c2OC(C)(COc3ccc(CC4SC(=O)NC4=O)cc3)CCc2...,"['[C]', '[C]', '[=C]', '[Branch1_1]', '[C]', '...",0.3//-0.0//-0.0//0.1//0.0//-0.1//-0.0//-0.1//-...
4,MCF7,10 uM,BRD-K41260949,CCCC(CCC)C(O)=O,"['[C]', '[C]', '[C]', '[C]', '[Branch1_1]', '[...",-0.1//-0.3//-0.0//0.1//-0.0//0.1//0.1//0.1//0....



Columns: ['cell_line', 'dose', 'drug_id', 'smiles', 'selfies', 'gene_e']
